<a href="https://colab.research.google.com/github/Himanshupandey1314/localrepo/blob/main/minor_project_2_backtesting_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [2]:
pip install pyngrok

In [3]:
!pip install vectorbt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.8 MB/s eta 0:00:00
  Created wheel for vectorbt: filename=vectorbt-0.26.2-py3-none-any.whl size=527326 sha256=1b4140e57606a628a52b6ba304c0519c188ba21a80a0c2bbf8dc1d3146230d0b
  Stored in directory: /root/.cache/pip/wheels/35/a3/d2/ae1a8cbf659fffc7d1890123df1a7fa2e7f6df587c1dca71cd
Successfully built vectorbt
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      

In [1]:
%%writefile app.py
import streamlit as st
import vectorbt as vbt
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from datetime import datetime
import pytz

# Function to convert date to timezone-aware datetime
def convert_to_timezone_aware(date_obj):
    return datetime.combine(date_obj, datetime.min.time()).replace(tzinfo=pytz.UTC)

# Streamlit app configuration
st.set_page_config(page_title="Backtesting Engine", layout="wide")
st.title("Backtesting Engine")

# Sidebar for user inputs
with st.sidebar:
    st.header("Strategy Settings")

    # Symbol and date inputs
    symbol = st.text_input("Enter the Symbol (e.g., 'AAPL')", value="AAPL")
    start_date = st.date_input("Start Date", value=pd.to_datetime("2015-01-01"))
    end_date = st.date_input("End Date", value=pd.to_datetime("2023-01-01"))

    # EMA period inputs
    short_ema_period = st.number_input("Short EMA Period", value=10, min_value=1)
    long_ema_period = st.number_input("Long EMA Period", value=20, min_value=1)

    st.header("Portfolio Settings")
    # Backtesting parameters
    initial_cash = st.number_input("Initial Cash", value=100000, step=1000)
    position_size = st.text_input("Position Size", value="50")
    size_type = st.selectbox("Size Type", ["amount", "value", "percent"], index=2)
    fees = st.number_input("Fees (in %)", value=0.1, format="%.4f")
    trade_direction = st.selectbox("Trade Direction", ["longonly", "shortonly", "both"], index=0)

    # Trigger for backtesting
    backtest = st.button("Run Backtest")

# Main content area
if backtest:
    # Convert dates to timezone-aware
    start_date_tz = convert_to_timezone_aware(start_date)
    end_date_tz = convert_to_timezone_aware(end_date)

    # Fetch historical data using VectorBT
    data = vbt.YFData.download(symbol, start=start_date_tz, end=end_date_tz).get('Close')

    # Calculate EMAs and trading signals
    short_ema = vbt.MA.run(data, short_ema_period, short_name="fast", ewm=True)
    long_ema = vbt.MA.run(data, long_ema_period, short_name="slow", ewm=True)
    entries = short_ema.ma_crossed_above(long_ema)
    exits = short_ema.ma_crossed_below(long_ema)

    # Convert position size based on size type
    size_value = float(position_size) / 100.0 if size_type == "percent" else float(position_size)

    # Backtesting the strategy
    portfolio = vbt.Portfolio.from_signals(
        data, entries, exits,
        direction=trade_direction,
        size=size_value,
        size_type=size_type,
        fees=fees / 100,
        init_cash=initial_cash,
        freq="1D"
    )

    # Creating tabs for different outputs
    tab1, tab2, tab3, tab4 = st.tabs(["Performance Stats", "Trade Records", "Equity Curve", "Drawdown"])

    # Tab 1: Performance stats
    with tab1:
        st.markdown("### Performance Metrics")
        stats = portfolio.stats()
        stats_df = pd.DataFrame(stats, columns=["Value"]).reset_index().rename(columns={"index": "Metric"})
        st.dataframe(stats_df)

    # Tab 2: Trade records
    with tab2:
        st.markdown("### Trade Records")
        trades = portfolio.trades.records_readable
        trades = trades.round(2)
        st.dataframe(trades)

    # Tab 3: Equity curve
    with tab3:
        equity_curve = portfolio.value()
        fig = go.Figure(data=go.Scatter(x=equity_curve.index, y=equity_curve, mode="lines", name="Equity"))
        fig.update_layout(title="Equity Curve", xaxis_title="Date", yaxis_title="Portfolio Value")
        st.plotly_chart(fig)

    # Tab 4: Drawdown
    with tab4:
        drawdown = portfolio.drawdown() * 100
        fig = go.Figure(data=go.Scatter(x=drawdown.index, y=drawdown, fill="tozeroy", mode="lines", name="Drawdown"))
        fig.update_layout(title="Drawdown", xaxis_title="Date", yaxis_title="Drawdown (%)")
        st.plotly_chart(fig)


Writing app.py


In [2]:
 ! curl 4.icanhazip.com

35.236.156.24


In [3]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
added 22 packages in 5s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [4]:
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.236.156.24:8501

your url is: https://red-shirts-attack.loca.lt
2024-12-10 09:42:09.421 Serialization of dataframe to Arrow table was unsuccessful due to: ("object of type <class 'pandas._libs.tslibs.timedeltas.Timedelta'> cannot be converted to int", 'Conversion failed for column Value with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
2024-12-10 09:42:29.461 Serialization of dataframe to Arrow table was unsuccessful due to: ("object of type <class 'pandas._libs.tslibs.timedeltas.Timedelta'> cannot be converted to int", 'Conversion failed for column Value with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
2024-12-10 09:44:02.987 Serialization of dataframe to Arrow table was unsuccessful due to: ("object of type <class 'pa